## Multimodal clustering 



In [9]:
path = "" # your path here 

import pandas as pd 
data = pd.read_csv(f"{path}gymshark_data.csv")
data.head()

,title,image_url,gender
0,"Arrival 5"" Shorts",https://cdn.shopify.com/s/files/1/0156/6146/fi...,1
1,2012 Lifting Hoodie,https://cdn.shopify.com/s/files/1/0156/6146/fi...,1
2,Minimal Sports Bra,https://cdn.shopify.com/s/files/1/0156/6146/fi...,0
3,Strength Department Graphic Joggers,https://cdn.shopify.com/s/files/1/0156/6146/fi...,0
4,Arrival Shorts,https://cdn.shopify.com/s/files/1/0156/6146/pr...,1


Using the above dataframe, I used the image urls downloaded each image to my local device, and then I made a new dataframe that contained the paths of all these images. However, seeing that I am not uploading the images to github, and the fact that the dataframe contains personal information, I have not included it. Please contact me if you need code on how to create this dataframe yourself. 

Basically you have to go through the above dataframe, download each image using the image_url, and then create a new column with the path to each respective image. I called this new dataframe "gymshark_data_with_image_paths.csv". 

code available here: https://docs.google.com/document/d/1Ocl57rQBJn0R4koIy3iB2aprEJbWyrlORK01-zob6JQ/edit?tab=t.0 

In [8]:
data = pd.read_csv(f"{path}/gymshark_data_with_image_paths.csv")
data = data.drop("image", axis=1)

# K-means clustering 
### 1. Early fusion with K-means clustering 

In [13]:
# let's try k-means, early fusion, semantic segmentation 

import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from torchvision import transforms
from sklearn.cluster import KMeans
from PIL import Image

In [ ]:
# tokenize the text descriptions 

# we use small / basic bert model because we don't need anything too complex. 
# we use uncased because capitalization doesn't matter for our task. 
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertModel.from_pretrained('bert-base-uncased')

def get_text_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=50)
    outputs = model(**inputs)
    # return embeddings as a single flattened vector 
    return outputs.last_hidden_state.mean(dim=1).detach().numpy().flatten()


In [ ]:
import pickle 

with open('bert_model.pkl', 'wb') as f:
    pickle.dump({'tokenizer': tokenizer, 'model': model}, f)

In [ ]:
with open('bert_model.pkl', 'rb') as f:
    saved_data = pickle.load(f)
    tokenizer = saved_data['tokenizer']
    model = saved_data['model']

In [ ]:
# crop out faces (these reveal the gender!)
import cv2

def crop_face(image_path): 
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml') 
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in faces:
        image[y:y+h, x:x+w] = (0, 0, 0)
    return image 


In [ ]:
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),   
    transforms.ToTensor(),           
    transforms.Normalize(           
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    ),
])

# image is in cv2.imread format 
def get_image_embedding(image):
    #image = Image.open(image_path)
    image = image_transform(image).unsqueeze(0)  
    with torch.no_grad():
        embedding = resnet_model(image)  
    return embedding.flatten().numpy() 

In [10]:
import pickle 

with open('resnet_model.pkl', 'wb') as f:
    pickle.dump(resnet_model, f)


"import pickle \n\nwith open('resnet_model.pkl', 'wb') as f:\n    pickle.dump(resnet_model, f)\n"

In [ ]:
with open('resnet_model.pkl', 'rb') as f:
    resnet_model = pickle.load(f)

# ensure the loaded model is in evaluation mode
resnet_model.eval()

In [ ]:
embeddings = []
texts = data['title']
images = data['image_path']

import cv2 

face_cascade = cv2.CascadeClassifier(f'{path}gymshark/cascades/haarcascade_frontalface_alt2.xml')

def crop_face(image_path): 
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    height, width = image.shape[:2]
    for (x, y, w, h) in faces:
        top = max(y - 70, 0)
        bottom = min(y + h + 70, height)
        left = max(x - 70, 0)
        right = min(x + w + 70, width)
        image[top:bottom, left:right] = (0, 0, 0)
    return image 


image_transform = transforms.Compose([
    transforms.Resize((224, 224)),   
    transforms.ToTensor(),          
    transforms.Normalize(            
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    ),
])

# image is in cv2.imread format 
def get_image_embedding(image):
    #image = Image.open(image_path)
    # Convert BGR image (from cv2) to RGB for PIL compatibility
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    image = image_transform(image).unsqueeze(0)  # Apply transformation and add batch dimension
    with torch.no_grad():
        embedding = resnet_model(image)  # Forward pass to get the feature vector
    return embedding.flatten().numpy()  # Flatten the tensor to a 1D array


for i, text in enumerate(texts):
    text_embedding = get_text_embedding(text)
    face_cropped_image = crop_face(images[i])
    image_embedding = get_image_embedding(face_cropped_image)
    combined_embedding = np.concatenate([text_embedding, image_embedding])
    embeddings.append(combined_embedding)

with open('embeddings.pkl', 'wb') as f: 
    pickle.dump(embeddings, f)

In [14]:
import pickle
import numpy as np 
with open('embeddings.pkl', 'rb') as f:
    embeddings_loaded = pickle.load(f)

# ensure it's the proper format before applying k-means clustering! 
embeddings_loaded = np.array(embeddings_loaded)

In [ ]:
# TESTING OUT IMAGE CROPPING 
image = cv2.imread(data.loc[2,'image_path'])
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.1, 4)
height, width = image.shape[:2]
for (x, y, w, h) in faces:
    top = max(y - 70, 0)
    bottom = min(y + h + 70, height)
    left = max(x - 70, 0)
    right = min(x + w + 70, width)
    image[top:bottom, left:right] = (0, 0, 0)
cv2.imwrite("image_2_cropped.jpg", image)

True

In [44]:
# Apply KMeans clustering with k=9
cluster_num = 6
kmeans = KMeans(n_clusters=6, random_state=42)
kmeans.fit(embeddings_loaded)

# Get the cluster labels
cluster_labels = kmeans.labels_

In [12]:
data = pd.read_csv("gymshark_data_with_image_paths.csv")
data = data.rename(columns={"title":"Product", "image_url":"image_url", "gender":"Gender"})
data = data.drop("image", axis=1)
embeddings = []
texts = data['Product']
images = data['image_path']


data['cluster'] = None 
for i, text in enumerate(texts):  
    data.loc[i, 'cluster'] = cluster_labels[i]


In [18]:
import random 
grouped = data.groupby('cluster')
# change this depending on number of clusters 
for cluster_number in [1,2, 3]:
    cluster_data = grouped.get_group(cluster_number)
    if len(cluster_data) >= 5: 
        sample = cluster_data.sample(n=5)
    else:
        sample = cluster_data
    
    print(f"\nCluster {cluster_number} Sample:")
    for _, row in sample.iterrows():
        print(row['title'], row['gender'])  


Cluster 1 Sample:
Strength Department Oversized Hoodie 0
Training T-Shirt 1
Arrival Regular Fit T-Shirt 1
Crest Oversized Hoodie 1
Training Oversized T-Shirt 0

Cluster 2 Sample:
Minimal Sports Bra 0
Adapt Safari Seamless Bralette 0
Minimal Sports Bra 0
Ruched Sports Bra 0
Cotton Lifting Sports Bra 0

Cluster 3 Sample:
Training Leggings 0
Training Fleece Joggers 0
Vital Seamless 2.0 Leggings 0
Vital Seamless 2.0 Leggings 0
2012 Lifting Joggers 1


In [10]:
for cluster_number in [4, 5]:
    cluster_data = grouped.get_group(cluster_number)
    if len(cluster_data) >= 5: 
        sample = cluster_data.sample(n=5)
    else:
        sample = cluster_data
    
    print(f"\nCluster {cluster_number} Sample:")
    for _, row in sample.iterrows():
        print(row['title'], row['gender']) 


Cluster 4 Sample:
Legacy T-Shirt 1
Strength Department Brushed Sweatshirt 0
Heritage Washed Hoodie 1
Strength Department Brushed Sweatshirt 0
Power T-Shirt 1

Cluster 5 Sample:
Arrival Shorts 1
Arrival 5" Shorts 1
Arrival 5" Shorts 1
Everyday Seamless Shorts 0
Adapt Safari Tight Shorts 0


Looking pretty good! these clusters seem to resemble cohesive clothing categories such as shirts, shorts, sweaters, bras, and pants!! 

### 2. Late fusion with K means clustering 

https://ieeexplore.ieee.org/document/8673554 
experiment with different weights for late fusion 

https://ieeexplore.ieee.org/abstract/document/10185035 
states that early fusion should work better. 

In [38]:
# separate each embedding from image and text esp if you don't want to run all that again 

text_embeddings = []
image_embeddings = []

# assuming positive numbers 
def round_normal(floatie): 
    return (int(floatie)+1) if floatie - int(floatie) >= 0.5 else int(floatie)

i = 0
while i < len(embeddings_loaded): 
    current_embedding = embeddings_loaded[i]
    text_embeddings.append(np.array(current_embedding[:768]))
    image_embeddings.append(np.array(current_embedding[768:]))
    i = i+1 

In [13]:
# run k-means clustering on each 

# you can edit these params 
cluster_num = 6
weight_for_image = 0.99 #weight for text automatically set to 1-(weight for image)
weight_for_text = 1-weight_for_image

kmeans_text = KMeans(n_clusters=cluster_num, random_state=42)
kmeans_text.fit(text_embeddings)
cluster_label_text =  kmeans_text.labels_

kmeans_images = KMeans(n_clusters=cluster_num, random_state=42)
kmeans_images.fit(image_embeddings)
cluster_label_images = kmeans_images.labels_

data['late_fusion_cluster'] = None

i = 0 
for cluster_pair in zip(cluster_label_text, cluster_label_images): 
    modified_cluster_val = (weight_for_image)*cluster_label_images[i] + (weight_for_text)*cluster_label_text[i]
    data.loc[i, 'late_fusion_cluster'] = round_normal(modified_cluster_val)
    i = i+1 


In [53]:
grouped = data.groupby('late_fusion_cluster')
for cluster_number in [1,2,3]:
    cluster_data = grouped.get_group(cluster_number)
    if len(cluster_data) >= 5: 
        sample = cluster_data.sample(n=5)
    else:
        sample = cluster_data
    
    print(f"\nCluster {cluster_number} Sample:")
    for _, row in sample.iterrows():
        print(row['title'], row['gender'])  


Cluster 1 Sample:
Strength Department Oversized Hoodie 0
GS Power Oversized T-Shirt 0
Arrival Regular Fit T-Shirt 1
Impact Drop Arm Tank 1
Lifting Essentials Oversized Sweatshirt 0

Cluster 2 Sample:
Minimal Sports Bra 0
Everyday Seamless Sports Bra 0
One Shoulder Sports Bra 0
Minimal Sports Bra 0
Everyday Seamless Sports Bra 0

Cluster 3 Sample:
2012 Lifting Joggers 1
Vital Seamless 2.0 Leggings 0
Adapt Camo Seamless Leggings 0
Everyday Seamless Leggings 0
Elevate Leggings 0


In [55]:
print(data['late_fusion_cluster'].unique())
for cluster_number in [4, 5]:
    cluster_data = grouped.get_group(cluster_number)
    if len(cluster_data) >= 5: 
        sample = cluster_data.sample(n=5)
    else:
        sample = cluster_data
    
    print(f"\nCluster {cluster_number} Sample:")
    for _, row in sample.iterrows():
        print(row['title'], row['gender']) 

[0 1 2 3 4 5]

Cluster 4 Sample:
Rest Day Essential Crew 1
Training Oversized Fleece Sweatshirt 0
Crest Hoodie 1
Rest Day Essential Crew 1
Rest Day Essential Crew 1

Cluster 5 Sample:
Graphic Crew Socks 7PK 0
Graphic Crew Socks 7PK 0
Graphic Crew Socks 7PK 0
Graphic Crew Socks 7PK 0
Graphic Crew Socks 7PK 0


These clusters seem okay too. Theoretiically tho early fusion is better for my type of problem (since we don't have missing data) so I think i'll just stick with the results of early fusion. 

## FINAL PRICING ANALYSIS 

Let's use the clusters determined by early fusion. 

In [16]:
import pandas as pd 
import statistics

def round_normal(floatie): 
    return (int(floatie)+1) if floatie - int(floatie) >= 0.5 else int(floatie)

In [14]:
price_df = pd.read_csv("gymshark_data_with_prices.csv")
price_df_dict = dict(
    zip(
        zip(price_df)
    )

)
grouped = price_df.groupby(["Product", "Gender"])["Price"].mean()
price_dict = grouped.to_dict()
grouped2 = data.groupby(["Product", "Gender"])["cluster"].mean()
data_dict = grouped2.to_dict()
price_a = pd.DataFrame(columns=["Cluster", "Average womens price", "Average mens price"])

# change number of lists based on how many clusters you ran 
cluster1_w = []
cluster1_m = []
cluster2_w = []
cluster2_m = []
cluster3_w = []
cluster3_m = []
cluster4_w = []
cluster4_m = []
cluster5_w = []
cluster5_m = []


cluster_dict = {f"cluster{i}_{gender}": [] for i in range(0, 7) for gender in ['w', 'm']}

for product_tuple, price in price_dict.items(): 
    if product_tuple in data_dict.keys(): 
        cluster = round_normal(data_dict[product_tuple])
        gender = 'w' if product_tuple[1] == 0 else 'm'
        list_name = f"cluster{cluster}_{gender}"
        cluster_dict[list_name].append(price)

i = 0 
for cat_name, prices in cluster_dict.items(): 
    cluster = cat_name[7]
    gender = "womens" if cat_name[9] == 0 else "mens"
    avg_price = statistics.mean(prices)
    if gender == 0: 
        price_a.loc[i] = [cluster, ]
cluster_list = list(cluster_dict.items())
i = 0 
while i < len(cluster_list): 
    if i % 2 == 1: 
        cluster_name = cluster_list[i][0]
        cluster = cluster_name[7]
        list_of_prices = cluster_list[i][1]
        avg_price_m = statistics.mean(list_of_prices) if len(list_of_prices) > 0 else 0 
        index = (i-1)/2
        price_a.loc[index] = [cluster, avg_price_w, avg_price_m]
        avg_price_m = '20000' 
        avg_price_w = '20000'
    if i % 2 == 0: 
        list_of_prices = cluster_list[i][1]
        avg_price_w = statistics.mean(list_of_prices) if len(list_of_prices) > 0 else 0 
    i+=1 


In [129]:
price_a = price_a.drop_duplicates(subset=["Cluster"])
price_a = price_a.sort_values(by='Cluster', ascending=True)
print(price_a)

     Cluster  Average womens price  Average mens price
0.0        0             32.000000            0.000000
1.0        1             39.373333           27.851832
2.0        2             30.500000            0.000000
3.0        3             48.325000           43.529167
9.0        4             37.744000           37.928571
5.0        5             40.000000           24.950000
13.0       6              0.000000            0.000000


# And voila! 
After lots of experimenting, we have finally created an automated approach to 
1. Creating cohesive and meaningful clothing categories across gender (5-6 clusters is a good number)
2. Comparing the price across categories to quantify any pricing discrimination! 

Looks like  multimodal AI saved the day. 